In [2]:
import numpy as np
from itertools import combinations
import networkx as nx
from collections import defaultdict
import os



def generate_possible_pairs(p, e, b):
    polish_only = list(range(1, p + 1))
    english_only = list(range(p + 1, p + e + 1))
    bilingual = list(range(p + e + 1, p + e + b + 1))

    possible_pairs = []

    # Polish-only pairs
    possible_pairs.extend(combinations(polish_only, 2))
    # English-only pairs
    possible_pairs.extend(combinations(english_only, 2))
    # Polish and bilingual
    possible_pairs.extend([(po, bi) for po in polish_only for bi in bilingual])
    # English and bilingual
    possible_pairs.extend([(eo, bi) for eo in english_only for bi in bilingual])
    # Bilingual pairs
    possible_pairs.extend(combinations(bilingual, 2))

    return possible_pairs

def max_rounds(p, e, b):
    possible_pairs = generate_possible_pairs(p, e, b)

    G = nx.Graph()
    G.add_edges_from(possible_pairs)

    rounds = 0
    total_people = p + e + b
    if total_people % 2 != 0:
        return 0, []  # If odd number of people, cannot pair everyone in each round

    all_round_pairs = []
    
    while True:
        matching = nx.max_weight_matching(G, maxcardinality=True)
        if len(matching) * 2 != total_people:
            break  # If we can't pair everyone, stop the rounds

        # Convert set of pairs to a sorted list of tuples for consistent output
        round_pairs = sorted(list(matching))
        all_round_pairs.append(round_pairs)
        print(f"Round {rounds + 1}: {round_pairs}")

        # Remove the matched pairs from the graph
        G.remove_edges_from(matching)
        
        rounds += 1

    return rounds, all_round_pairs

def assign_tables_to_rounds(round_pairs):
    table_assignments = []
    n_tables = len(round_pairs[0])  # Determine number of tables from the first round
    
    for round_num, pairs in enumerate(round_pairs, start=1):
        table_assignment = {}
        for table_number, pair in enumerate(pairs, start=1):
            table_assignment[pair] = table_number
        table_assignments.append(table_assignment)
    return table_assignments

def generate_visible_ids(p, e, b):
    polish_only = [f"P{i:03}" for i in range(1, p + 1)]
    english_only = [f"E{i:03}" for i in range(1, e + 1)]
    bilingual = [f"B{i:03}" for i in range(1, b + 1)]
    visible_ids = polish_only+english_only+bilingual

    return {id:visible_id for id, visible_id in zip(range(1,p+e+b+1), visible_ids)}


def generate_seating_guides(round_pairs, table_assignments):
    participant_info = defaultdict(list)
    for round_number, (pairs, table_assignment) in enumerate(zip(round_pairs, table_assignments), start=1):
        for pair in pairs:
            table_number = table_assignment[pair]
            participant_info[pair[0]].append({'round':round_number, 'table':table_number})
            participant_info[pair[1]].append({'round':round_number, 'table':table_number})
    return participant_info

def generate_html_files(participant_info, visible_ids, n_rounds=10):
    os.makedirs('participants', exist_ok=True)

    for participant_id, info_list in participant_info.items():
        with open(f'participants/{visible_ids[participant_id]}.html', 'w') as f:
            f.write(f"<html><body><h1>{visible_ids[participant_id]}</h1>\n")

            if visible_ids[participant_id][0]!="E":
                for i in range(n_rounds):
                    r=info_list[i]['round']
                    t=info_list[i]['table']
                    f.write(f"<p>runda {r}: stolik {t}</p>\n")
            else:
                for i in range(n_rounds):
                    r=info_list[i]['round']
                    t=info_list[i]['table']
                    f.write(f"<p>round {r}: table {t}</p>\n")



            f.write("</body></html>")





In [4]:
p = 1  # number of polish-only speakers
e =  1 # number of english-only speakers
b =  4 # number of bilingual speakers

rounds, round_pairs = max_rounds(p, e, b)
table_assignments = assign_tables_to_rounds(round_pairs)
participant_info = generate_seating_guides(round_pairs, table_assignments)


# Output participant info
for participant_id, info in participant_info.items():
    print(f"Participant {participant_id}:\n" + "\n"+str(info) + "\n")
generate_html_files(participant_info, generate_visible_ids(p,e,b), n_rounds=3)


Round 1: [(2, 3), (5, 4), (6, 1)]
Round 2: [(2, 4), (5, 1), (6, 3)]
Round 3: [(2, 5), (3, 1), (6, 4)]
Round 4: [(2, 6), (4, 1), (5, 3)]
Participant 2:

[{'round': 1, 'table': 1}, {'round': 2, 'table': 1}, {'round': 3, 'table': 1}, {'round': 4, 'table': 1}]

Participant 3:

[{'round': 1, 'table': 1}, {'round': 2, 'table': 3}, {'round': 3, 'table': 2}, {'round': 4, 'table': 3}]

Participant 5:

[{'round': 1, 'table': 2}, {'round': 2, 'table': 2}, {'round': 3, 'table': 1}, {'round': 4, 'table': 3}]

Participant 4:

[{'round': 1, 'table': 2}, {'round': 2, 'table': 1}, {'round': 3, 'table': 3}, {'round': 4, 'table': 2}]

Participant 6:

[{'round': 1, 'table': 3}, {'round': 2, 'table': 3}, {'round': 3, 'table': 3}, {'round': 4, 'table': 1}]

Participant 1:

[{'round': 1, 'table': 3}, {'round': 2, 'table': 2}, {'round': 3, 'table': 2}, {'round': 4, 'table': 2}]

